# Tutorial 13-02 - Create a Python Toolbox

Now let's create a Python toolbox.  

**TODO** ADD CONTEXT

## Create a Template Python Toolbox

#### 1.  Open ArcGIS Pro.

If you don't already have ArcGIS Pro open, open the project for this chapter.

#### 2.  Navigate to the Catalog View

Open the Catalog View or Pane.  

#### 3.  Right-click on the Toolboxes folder

In the **Contents** pane of the Catalog View, under the **Project** section, right-click on the **Toolboxes** folder.

#### 4.  Create a new Python Toolbox

Click on **New Python Toolbox**.

![Create a new Python Toolbox.](resources\new_python_toolbox.png "Create a new Python Toolbox")

This will bring up a window allowing you to navigate to a folder and save the toolbox.  By default, the toolbox will save to your project folder.  You can choose a name for the toolbox.  In this example, we're going to call the toolbox *Data Export Tools*.  When you click **Save**, that window will close and you'll see a new file in your toolboxes called *Data Export Tools.pyt*.

## Explore the Template

#### 1.  Open the Python Toolbox in an editor

Right-click your new Python toolbox and click **Edit**.  This will open the toolbox in your Python editor of choice.

#### 2.  Review the template code

The template code here might look intimidating at first but it's not too complex if you examine its components.  The template for a python toolbox comes with two classes.  

The first class is the `Toolbox`.  This class defines the toolbox itself and has descriptive information about the toolbox.  You can set a label and alias here.  You'll also need to use the `self.tools` property later to ensure that your tools are listed in the toolbox.  You'll end up modifying this class in place.

The second class, `Tool`, is an example tool.  This class has several different methods that execute at different points during the execution cycle of a tool.  
 - `__init__` - Defines how the tool appears in the toolbox.  This method initializes the tool and contains metadata.
 - `getParameterInfo` - This is where you'll define your parameters, which will determine how your users can use this tool.
 - `isLicensed` - This method allows you to insert custom logic to determine whether the tool can be run.  For instance, you could check to ensure that your user has a specific extension of ArcGIS Pro here.
 - `updateParameters` - This method is called whenever a user populates a parameter.  You can insert custom logic in here to update other parameters based on what a user has already entered.
 - `updateMessages` - This method is called after internal validation and is a place where you can raise parameter-related errors to the user.
 - `execute` - This is where the body of your code will go.  All the logic we put into the main function of our previous tool will go here.
 - `postExecute` - This executes after the body of your code has executed and any outputs have been added into the display/map.  Often this is used for symbology and redering changes.

In [1]:
# -*- coding: utf-8 -*-

import arcpy


class Toolbox:
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [Tool]


class Tool:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Tool"
        self.description = ""

    def getParameterInfo(self):
        """Define the tool parameters."""
        params = None
        return params

    def isLicensed(self):
        """Set whether the tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter. This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return


## Write A Tool

Now that you've got your template set up, you can start writing tools.  Each new tool you write will be its own class.

#### 1.  Copy the template Tool class

The easiest way to get started creating a tool is to copy the template `Tool` class.  Copy that class and all its methods and paste at the bottom of your .pyt file.

#### 2.  Change the name and description of your new tool

You can start customizing with the class name and the `__init__` method.  

Change the class name from "Tool" to "CountyExporter".

In the `__init__` method, change the following:
 - `self.label = "County Highway Exporter"`
 - `self.description = "This tool exports a zipped file geodatabase of highway data"`

In [2]:
class CountyExporter:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "County Highway Exporter"
        self.description = "This tool exports a zipped file geodatabase of highway data"


#### 3.  Create parameters.

Similarly to the previous tutorial, you'll need to create parameters to define your user interaction. In a Python toolbox, you define those parameters using code.  You'll use the `arcpy.Parameter` class to create these parameters.

Each parameter will need the same information that you filled out in the Parameter panel in the previous tutorial.  Below is a sample of what an individual parameter looks like.

In [ ]:
full_fc_path = arcpy.Parameter(
    displayName="Highways Feature Class",
    name="Highways_Feature_Class",
    datatype="GPFeatureLayer",
    parameterType="Required",
    direction="Input"
)

Note that the properties of this parameter are very similar to the GUI-based workflow in the previous tutorial.  For each parameter, you'll specify the display name, the machine-friendly name, the data type, the parameter type, and direction.

TIP - The data types for parameters in a python toolbox have very specific names.  It's helpful to look up the GP Data Types when writing parameters.

Update the `getParameterInfo()` method to include the following parameters.  Make sure to put all your parameter objects in the `params` list that gets returned by this method.

In [1]:
    def getParameterInfo(self):
        """Define the tool parameters."""
        # First parameter
        full_fc_path = arcpy.Parameter(
            displayName="Highways Feature Class",
            name="Highways_Feature_Class",
            datatype="GPFeatureLayer",
            parameterType="Required",
            direction="Input")

        # Second parameter
        output_folder = arcpy.Parameter(
            displayName="Output Folder",
            name="Output_Folder",
            datatype="DEFolder",
            parameterType="Required",
            direction="Input")


        # Third parameter
        county = arcpy.Parameter(
            displayName="County",
            name="County",
            datatype="GPString",
            parameterType="Required",
            direction="Input")
        
        # list of all parameters
        params = [full_fc_path, output_folder, county]
        return params

#### 4.  Paste the function you developed into the .pyt file

Because you can reference all the code in your .pyt file, you can paste your pre-defined function into the file and reference it during execution.  For reference, here is the function.  You can paste below all the other code.  Make sure it's fully left-justified and not tabbed in at all (so that it's a function and not a method of one of the objects).

In [2]:
def zip_county_highways(full_fc_path, output_folder, county):

    # remove spaces from county name
    county_no_spaces = county.replace(" ", "_")
    
    # create a file geodatabase
    fgdb = arcpy.management.CreateFileGDB(
        out_folder_path = output_folder,
        out_name = f"{county_no_spaces}_Output"
    )

    # Create a feature class
    output_fc = arcpy.conversion.ExportFeatures(
        in_features = full_fc_path,
        out_features = os.path.join(fgdb[0], 
                                    f"{county_no_spaces}_Highways"),
        where_clause = f"NAMELSAD = '{county}'"
    )
    
    # define a path for the zip file
    zip_file_path = os.path.join(output_folder, f"{county} Highways.zip") 

    # zip the file geodatabase
    with zipfile.ZipFile(zip_file_path, "w") as zipper:
        for root, dirs, files in os.walk(fgdb[0]):
            for file in files:
                fpath = os.path.join(root, file)
                zpath = os.path.relpath(
                            os.path.join(root, file),
                            os.path.join(fgdb[0], '..')
                        )
                zipper.write(
                    fpath,
                    zpath
                )
    
    # delete the file geodatabase
    arcpy.management.Delete(fgdb)
    
    # return the zip file path
    return zip_file_path

#### 5.  Write the execution portion of your tool

Now that you've set up your parameters and inserted the function that defines your tool logic, you can populate the `execute` method of your tool.  

The `execute` method accepts the list of parameters that is returned by the `getParameterInfo` method.  You can access those parameters by index in the order you returned them in the `getParameterInfo` method.  You'll need to call the `valueAsText` property on each parameter in this case.

In [ ]:
highways_feature_class = parameters[0].valueAsText
output_folder = parameters[1].valueAsText
county = parameters[2].valueAsText

Now that you've unpacked the parameters, you can call the function you defined in the previous step.

In [ ]:
zip_county_highways(
    highways_feature_class, 
    output_folder,
    county
)

When you put all that together in the `execute` method, it should look like this

In [4]:
    def execute(self, parameters, messages):
        """The source code of the tool."""
        
        highways_feature_class = parameters[0].valueAsText
        output_folder = parameters[1].valueAsText
        county = parameters[2].valueAsText

        zip_county_highways(
                highways_feature_class, 
                output_folder,
                county
            )
        return

#### 6.  Add your tool to the toolbox

Your tool should now be functional, but if you were to save now and look at the toolbox in ArcGIS Pro, you wouldn't see your new tool.  This is because you have to add your tool object to the toolbox's tool list.

Find the `tools` parameter of the toolbox and add your tool ("CountyExporter") to that list.  Your toolbox class should look like this now.

In [6]:
class Toolbox:
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [Tool, CountyExporter]

## Test Your Tool

#### 1.  Review your code.

Now that you've set up your tool, defined your parameters, and defined the execution of your tool, you can test the tool.  Save your .pyt file.  The code should look like the following code block.

In [7]:
# -*- coding: utf-8 -*-

import arcpy
import zipfile
import os


class Toolbox:
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Toolbox"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [Tool, CountyExporter]


class Tool:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "Tool"
        self.description = ""

    def getParameterInfo(self):
        """Define the tool parameters."""
        params = None
        return params

    def isLicensed(self):
        """Set whether the tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter. This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return


class CountyExporter:
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "County Highway Exporter"
        self.description = "This tool exports a zipped file geodatabase of highway data"

    def getParameterInfo(self):
        """Define the tool parameters."""
        # First parameter
        full_fc_path = arcpy.Parameter(
            displayName="Highways Feature Class",
            name="Highways_Feature_Class",
            datatype="GPFeatureLayer",
            parameterType="Required",
            direction="Input")

        # Second parameter
        output_folder = arcpy.Parameter(
            displayName="Output Folder",
            name="Output_Folder",
            datatype="DEFolder",
            parameterType="Required",
            direction="Input")


        # Third parameter
        county = arcpy.Parameter(
            displayName="County",
            name="County",
            datatype="GPString",
            parameterType="Required",
            direction="Input")
        
        params = [full_fc_path, output_folder, county]
        return params

    def isLicensed(self):
        """Set whether the tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter. This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        
        highways_feature_class = parameters[0].valueAsText
        output_folder = parameters[1].valueAsText
        county = parameters[2].valueAsText

        zip_county_highways(
                highways_feature_class, 
                output_folder,
                county
            )
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

def zip_county_highways(full_fc_path, output_folder, county):

    # remove spaces from county name
    county_no_spaces = county.replace(" ", "_")
    
    # create a file geodatabase
    fgdb = arcpy.management.CreateFileGDB(
        out_folder_path = output_folder,
        out_name = f"{county_no_spaces}_Output"
    )

    # Create a feature class
    output_fc = arcpy.conversion.ExportFeatures(
        in_features = full_fc_path,
        out_features = os.path.join(fgdb[0], 
                                    f"{county_no_spaces}_Highways"),
        where_clause = f"NAMELSAD = '{county}'"
    )
    
    # define a path for the zip file
    zip_file_path = os.path.join(output_folder, f"{county} Highways.zip") 

    # zip the file geodatabase
    with zipfile.ZipFile(zip_file_path, "w") as zipper:
        for root, dirs, files in os.walk(fgdb[0]):
            for file in files:
                fpath = os.path.join(root, file)
                zpath = os.path.relpath(
                            os.path.join(root, file),
                            os.path.join(fgdb[0], '..')
                        )
                zipper.write(
                    fpath,
                    zpath
                )
    
    # delete the file geodatabase
    arcpy.management.Delete(fgdb)
    
    # return the zip file path
    return zip_file_path




#### 2.  Refresh your toolbox in ArcGIS Pro

If you closed ArcGIS Pro at some point, re-open it.  If you had it open still, right click your python toolbox and click **Refresh**.  You should now see two tools in the toolbox.  One is the **Tool** tool that was in there originally.  Another is your new **County Highway Exporter** tool.

#### 3.  Open your new tool

Double-click the **Highways Exporter Tool** to open the tool in your Geoprocessing pane.

![Your new python tool.](resources\new_python_tool.png "Your new python tool")

#### 4.  Populate parameters to test with

Navigate to the files in the Chapter 3 (Arcpy Basics) of the tutorial files.  Find the file geodatabase included there.  Drag the "Highways_Intersect" feature class into the first parameter in your new tool.  Note how much easier it is to change these parameters than it is in a script.

Populate the following values in your new tool.
 - Highways Feature Class - *the path to the "Highways_Intersect" feature class*
 - Output Folder - *path to a folder on your computer where you can export a zip file*
 - County - **Alameda County**

#### 5.  Run your tool

Click **Run** in your new tool and export data.

If everything went well, your tool should have exported a zipped file geodatabase with a single county's highways included.

## Modify your Parameters

Assuming your tool is working correctly, there are some modifications you can make to make it a little easier on your end users.  Because you have control over the tool initialization and parameters, you can let your users choose from a pre-defined list of counties.  This will reduce the chance for user error and typos.

#### 1.  Open your python toolbox in edit mode

if you closed your toolbox, open it again by right-clicking it and clicking **Edit**.

#### 2.  Modify the county parameter.